### Imports

In [11]:

import tqdm as tqdm
from collections import Counter
import os
import pandas as pd
import re
from ast import literal_eval
import statistics
import matplotlib.pyplot as plt
import re
import pandas as pd

folder_path = "filtered/filtered_06_15 (only kw)"

### Data

In [12]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# redirect the working directory of this script to the data folder
%cd /content/drive/MyDrive/Work/Frontline/data/
#%cd /content/drive/MyDrive/data/

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [14]:


dfs = []

# loop through files
for filename in os.listdir(folder_path):
    # if csv file, load and add to dfs
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        # import csv with text as list object
        df = pd.read_csv(file_path, index_col=0, converters={"text":literal_eval})
        dfs.append(df)
# combine files in df
df_filtered = pd.concat(dfs, ignore_index=True)


In [15]:
df=pd.read_csv("filtered/filtered_06_16.csv",converters={"text":literal_eval})

In [16]:
df_test=df.iloc[:5000,:]

### 1. Sensationalism

In [17]:
from scripts import annotations

Looking for "sensationalist speech" in previously annotated Paragraphs

In [ ]:
ann=pd.read_csv("annotated/230621_all_annotationsV1.csv", index_col=0,converters={"annotations":literal_eval})
ann.loc[:,"annotations"]=ann.apply(annotations.ground_truth_filter,min_coannotation=1,min_similarity=-1, similarity="dice",axis=1)
sensationalist_texts=ann[ann.annotations=={"Sensationalist"}][["text"]]
sensationalist_texts

In [19]:
sens =["Ehetragödie",
       "verschmähter? Liebhaber(in)?",
       "rachsüchtige Hexe",
       "Eifersuchtstat",
       "Beziehungsdramas?",
       "Rosenkrieg",
       ##new
       "Frauenschläger",
       "tödliches?m? Drama",
       "tränenerstickter? Stimme",
       "blutüberströmt",
       "tödlichen? Eskalation",


       "ehedrama",
       ## ??
       "bluttat",

       "schicksalsgebeutelt",
       "Alptraum"]

In [20]:
def label_sensationalism(text):
  for pattern in sens:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Sensationalist"
  return ""

In [21]:
df_test=df_test.explode("text")

In [22]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_sensationalism)

### 2. Statistics

In [24]:
stats=["\d*([\.,]\d*)?[ ](Prozent|%)",
       "\d+ (Fälle|Opfer|Frauen|Kinder|Personen|Betroffene|Morde|Plätze)\W",
       "Bundeskriminalamt",
       "Kriminalstatistik",
       "statistisches Amt",
       "offizielle Zahlen",
       "jeder? (zweiter?|dritter?|vierter?|fünfter?|sechster?|siebenter?) (mann|frau|kind)",
       "\Wbka\W"]

In [25]:
def label_statistics(text):
  for pattern in stats:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Statistic"
  return ""

In [26]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_statistics)

In [ ]:
df_test[df_test["label"]=="Statistic"][["text"]].head()

### 3. Support Services

In [28]:
support=["[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}",
         '\(?\d{4,5}\)?[/\s]*\d{1,5}\s*\d{1,5}',
         "(www\.)\S+\.\S+",
         "Beratungsstelle",
         "Notrufdienst",
         "Hotline",
         ]

In [29]:
def label_support(text):
  for pattern in support:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Support Service"
  return ""

In [ ]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_support)
df_test[df_test["label"]=="Support Service"][["text"]]

### 4. Origin/ Nationality

In [32]:
nlp = spacy.load("de_core_news_lg")

In [33]:
nationality=["Afghan","Albaner","Algerier","Argentinier","Armenier", "Aserbaidschaner","Amerikaner","Argentinier", "Armenier","Aserbaidschaner","Australier", "Belgier","Brasilianer", "Bulgar","Chilen", "Chines","Dän","Deutsche", "Deutsche","Engländer", "Franzose", "Französin","Griech",
             "Inder","Iraner","Iraker","Israeli","Italiener","Japaner","Kanadier","Kolumbianer","Koreaner","Kroat","Libanes","Marokkaner","Mexikaner","Neuseeländer", "Niederländer",
             "Norweger","Österreicher", "Pakistaner", "Peruaner",
             #"Pol",
             "Portugies","Rumän","Russ","Schott","Schwed","Schweizer", "Serb","Singapurer", "Spanier","Südafrikaner",
             "Türk",]


In [34]:
def label_origin(text):
  for pattern in nationality:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Nationality"
  return ""

In [ ]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_origin)
df_test[df_test["label"]=="Nationality"][["text"]]

In [ ]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_origin)
df_test[df_test["label"]=="Nationality"][["text"]]

#### Testing POS instead

In [31]:
import spacy
spacy.cli.download("de_core_news_lg")

✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [56]:

doc = nlp(df_test.iloc[0]["text"])

for i in doc.ents:
  print(i.text,i.label_)

Göttingen LOC
Deutschen MISC
Hannover LOC


--> not usable to extraxt nationalities only